In [1]:
# Imports
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [14]:
# Load MNIST dataset from Keras
mnist = keras.datasets.mnist
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data() # load dataset

X_train_full.astype(np.float32)
y_train_full.astype(np.float32)
X_test.astype(np.float32)
y_test.astype(np.float32)

# Scale pixel intensities between 0-1 and create a validation set
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0 
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [3]:
# Create & compile simple sequential model
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(10, activation="softmax") # output
])

# Compile Model
model.compile(
    loss=keras.losses.sparse_categorical_crossentropy, # sparse labels
    optimizer=keras.optimizers.legacy.SGD(lr=0.01, clipvalue = 0.5),
    metrics=[keras.metrics.sparse_categorical_accuracy]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 dense_2 (Dense)             (None, 30)                3030      
                                                                 
 dense_3 (Dense)             (None, 10)                310       
                                                                 
Total params: 268940 (1.03 MB)
Trainable params: 268940 (1.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


/Users/alexjaniak/Programs/miniforge3/envs/zkMNIST/lib/python3.11/site-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [4]:
# Train Model
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid))

Epoch 1/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.6974 - sparse_categorical_accuracy: 0.8070 - val_loss: 0.2951 - val_sparse_categorical_accuracy: 0.9154
Epoch 2/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.2690 - sparse_categorical_accuracy: 0.9226 - val_loss: 0.2187 - val_sparse_categorical_accuracy: 0.9374
Epoch 3/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.2095 - sparse_categorical_accuracy: 0.9397 - val_loss: 0.1809 - val_sparse_categorical_accuracy: 0.9480
Epoch 4/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.1719 - sparse_categorical_accuracy: 0.9510 - val_loss: 0.1585 - val_sparse_categorical_accuracy: 0.9550
Epoch 5/30
1719/1719 [==============================] - 3s 2ms/step - loss: 0.1464 - sparse_categorical_accuracy: 0.9577 - val_loss: 0.1334 - val_sparse_categorical_accuracy: 0.9616
Epoch 6/30
1719/1719 [==============================] - 2s 1ms/step - loss: 0.1266 - spars

In [20]:
# Test
model.evaluate(X_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 15.4702 - sparse_categorical_accuracy: 0.9752


[15.470162391662598, 0.9751999974250793]

In [6]:
# Create new model without the output layer
frontend = keras.models.Sequential(model.layers[:-1])

# Copy the weights and biases from the original model to the new model
for layer, new_layer in zip(model.layers[:-1], frontend.layers):
    new_layer.set_weights(layer.get_weights())

frontend.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 dense_2 (Dense)             (None, 30)                3030      
                                                                 
Total params: 268630 (1.02 MB)
Trainable params: 268630 (1.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
# Manually compute the output of the new model using the last layer's weights and biases from the original model
last_layer_weights, last_layer_biases = model.layers[-1].get_weights()
frontend_output = tf.matmul(frontend.predict(X_valid[:2]), last_layer_weights) + last_layer_biases
frontend_output = tf.nn.softmax(frontend_output)

model_output = model.predict(X_valid[:2])

# Prepare the data for the table
table_data = [
    ["Original Model Output Shape:", model_output.shape],
    ["Original Model Output Values:", model_output],
    ["Original Model Prediction:", np.argmax(model_output, 1)],
    ["", ""],  # Empty row for spacing
    ["New Model Output Shape:", frontend_output.shape],
    ["New Model Output Values:", frontend_output.numpy()],
    ["New Model Prediction:", np.argmax(frontend_output, 1)],
    ["", ""],
    ["Expected Ouput:", y_valid[:2]]
]

# Determine the maximum length of the descriptions for alignment
max_desc_length = max(len(row[0]) for row in table_data)

# Print Test Results
print("-" * (max_desc_length + 2))
for row in table_data:
    description = row[0].ljust(max_desc_length)
    value = row[1]
    print(f"{description} {value}")
print("-" * (max_desc_length + 2))


print(frontend_output.numpy().all() == model_output.all())

1/1 [==============================] - 0s 19ms/step
-------------------------------
Original Model Output Shape:  (2, 10)
Original Model Output Values: [[8.7689613e-12 3.6768448e-08 1.7953114e-07 1.6368752e-02 3.4092315e-17
  9.8363101e-01 1.7702296e-12 2.6962912e-10 2.0212696e-11 1.0865001e-08]
 [9.9999619e-01 1.4972997e-12 3.5517605e-06 5.1023018e-12 5.6060295e-11
  2.4501759e-12 4.1206292e-08 1.9631443e-07 8.7891777e-11 1.6871030e-10]]
Original Model Prediction:    [5 0]
                              
New Model Output Shape:       (2, 10)
New Model Output Values:      [[8.7689613e-12 3.6768448e-08 1.7953114e-07 1.6368752e-02 3.4092315e-17
  9.8363101e-01 1.7702296e-12 2.6962912e-10 2.0212696e-11 1.0865001e-08]
 [9.9999619e-01 1.4972997e-12 3.5517605e-06 5.1023018e-12 5.6060295e-11
  2.4501759e-12 4.1206292e-08 1.9631443e-07 8.7891777e-11 1.6871030e-10]]
New Model Prediction:         [5 0]
                              
Expected Ouput:               [5 0]
----------------------------

In [8]:
model.layers[-1].get_weights()

[array([[-0.4746044 , -0.17401358,  0.04045997,  0.49065602,  0.34011176,
          0.37476963, -0.5948454 , -0.21871452, -0.4372033 ,  0.57128954],
        [ 0.32773218, -0.69739807,  0.47574416,  0.37404647, -0.90683013,
          0.13982633, -0.530216  ,  0.4522029 , -0.64315176, -0.04918684],
        [-0.02300171, -0.5227803 ,  0.46335906, -0.22812063, -0.11517239,
          0.03219993,  0.6284023 , -0.751931  ,  0.57648057,  0.03546061],
        [ 0.02755888,  0.7532413 ,  0.67983705, -0.02399273,  0.15439917,
         -0.3311136 ,  0.41011298, -0.00824951, -0.42705557, -0.86243606],
        [-0.32337558, -0.00815772,  0.49542132, -0.74162865,  0.6949287 ,
         -0.01529182,  0.2705112 ,  0.87966275, -0.251072  , -0.5111604 ],
        [ 0.853414  , -0.08771293, -0.3074586 , -0.58547544, -0.54565567,
         -0.7266197 ,  0.62607574,  0.4645215 , -0.56800693, -0.21421501],
        [-0.40069675, -0.45823002,  0.3586777 ,  0.7629971 ,  0.03995378,
          0.43127215,  0.3792151

In [9]:
# Manually Compute 

BIG_INT = 100000000 # future use: 2**32

# scale the input, weights, & baises to remove floats
scaled_input = frontend.predict(X_test[2:3])*1000 # input is positive due to ReLu
scaled_input = scaled_input.astype(int) # floor
#scaled_input += BIG_INT 

scaled_weights = last_layer_weights*1000
scaled_weights = scaled_weights.astype(int) # floor 
scaled_weights += BIG_INT

scaled_bias = last_layer_biases*1000*1000
scaled_bias = scaled_bias.astype(int)
scaled_bias += BIG_INT

output1 = tf.matmul(scaled_input, scaled_weights)
output2 = np.add(output1, scaled_bias)
output3 = np.argmax(output2)

print("scaled weights:", scaled_weights.transpose().flatten().tolist())
print("scaled input:", scaled_input.flatten().tolist())
print("scaled bias:", scaled_bias.tolist())
print("matmul output:", output1.numpy().astype(int).flatten().tolist())
print("output:", output2.astype(int).flatten().tolist())
print("argmax:", output3)

1/1 [==============================] - 0s 35ms/step
scaled weights: [99999526, 100000327, 99999977, 100000027, 99999677, 100000853, 99999600, 99999617, 99999675, 99999928, 100000629, 99999711, 99999426, 99999771, 99999384, 100000093, 100000149, 99999364, 99999845, 100000366, 99999940, 100000113, 100000521, 100000149, 100000024, 99999946, 99999509, 100000228, 100000349, 99999906, 99999826, 99999303, 99999478, 100000753, 99999992, 99999913, 99999542, 99999809, 100000317, 100000643, 99999245, 100000351, 100000510, 99999870, 99999823, 100000334, 100000091, 100000181, 99999702, 99999691, 99999967, 100000328, 100000362, 99999551, 100000765, 100000277, 100000019, 100000247, 100000674, 99999840, 100000040, 100000475, 100000463, 100000679, 100000495, 99999693, 100000358, 99999889, 99999556, 99999678, 99999978, 99999921, 100000270, 100000094, 99999446, 99999114, 100000336, 100000223, 100000239, 99999860, 100000121, 100000192, 100000262, 100000237, 100000888, 99999518, 99999506, 100000195, 999995

In [11]:
import tensorflowjs as tfjs

# Save Keras model for tfjs
tfjs.converters.save_keras_model(model, "model")

/Users/alexjaniak/Programs/miniforge3/envs/zkMNIST/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
# Save Keras model for model_test.ipynb
model.save("model/model.h5")

/Users/alexjaniak/Programs/miniforge3/envs/zkMNIST/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
